## __Tarea 2 - Parcial 1 "Georreferenciación Visual"__
### _Pasos para resolver la tarea_
- Corregir análisis de la tarea 1.
    - Abrir MySQL para ejecutar los scripts de SQL y crear las tablas con sus respectivos registros para posteriormente exportar estos datos en un archivo CSV.
    - Unir los dataframes por el código postal (simulando un VLOOKUP de Excel con Pandas en Python) y aplicar la fórmula de Haservine para obtener la distancia entre ambos lugares.
    - Filtrar por las distancias mayores a 10 Km.
- Visualización de los datos
    - Usando la librería Folium, se crea un documento html, el cual al ser abierto en el navegador, despliega un mapa con los puntos de las escuelas y colonias del dataset con los datos de las distancias mayores a 10 Km.




Se utilizó la tecnología "folium" para hacer la georreferenciación visual de los datos obtenidos después del análisis. Folium es una librería de python la cual permite hacer mapas y utilizar coordenadas extraídas de fuentes de datos para hacer representaciones visuales de ubicaciones de manera dinámica.

En el mapa se plasman los puntos de las ubicaciones de cada escuela y de cada colonia. Los puntos con el mismo CP se unen mediante una línea (La idea era que hacer las líneas dinámicas. Al seleccionar un punto se uniera con una línea a su punto complementario, pero para hacer eso se necesita de otras tecnologías como JavaScript y por temas de simplicidad y tiempo se descartó esta idea). En la parte final de este Jupyten Notebook se encuentra la sección donde se crea el mapa y se exporta a un documento html.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def haversine(lon1, lat1, lon2, lat2):
    #Convertir grados decimales a radianes
    lon1 = np.radians(lon1.astype(float))
    lat1 = np.radians(lat1.astype(float))
    lon2 = np.radians(lon2.astype(float))
    lat2 = np.radians(lat2.astype(float))

    #Formula de Haversine
    dis_lon = lon2 - lon1
    dis_lat  = lat2 - lat1
    a = np.sin(dis_lat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dis_lon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    #Radio de la tierra en KM
    r = 6371 

    return c * r

In [3]:
def clean_coord(val):
    if isinstance(val, str):
        val = val.replace(",", ".")  # si hay comas en lugar de punto decimal
    try:
        return float(val)
    except:
        return 0.0

In [4]:
# Cargar datos de colonias y renombrar las columnas
df_colonias = pd.read_csv('cat_colonias.csv')
df_colonias = df_colonias.rename(columns={
    "cve_codpost": "codigo_postal",
    "lat": "lat_colonias",
    "lon": "lon_colonias"
})

df_colonias["lat_colonias"] = pd.to_numeric(df_colonias["lat_colonias"], errors="coerce")
df_colonias["lon_colonias"] = pd.to_numeric(df_colonias["lon_colonias"], errors="coerce")
df_colonias['lat_colonias'] = df_colonias['lat_colonias'].fillna(0)
df_colonias['lon_colonias'] = df_colonias['lon_colonias'].fillna(0)

# Cargar datos de escuelas y renombrar las columnas
df_escuelas = pd.read_csv('edificios_escolares.csv')
df_escuelas = df_escuelas.rename(columns={'latitud': 'lat_escuelas', 'longitud': 'lon_escuelas'})

df_colonias["lat_colonias"] = pd.to_numeric(df_colonias["lat_colonias"], errors="coerce")
df_colonias["lon_colonias"] = pd.to_numeric(df_colonias["lon_colonias"], errors="coerce")
df_escuelas['lat_escuelas'] = df_escuelas['lat_escuelas'].fillna(0)
df_escuelas['lon_escuelas'] = df_escuelas['lon_escuelas'].fillna(0)

df_escuelas["lat_escuelas"] = df_escuelas["lat_escuelas"].apply(clean_coord)
df_escuelas["lon_escuelas"] = df_escuelas["lon_escuelas"].apply(clean_coord)

In [5]:
# Crear "diccionarios" de CP -> coordenadas
lat_lookup = df_colonias.drop_duplicates("codigo_postal").set_index("codigo_postal")["lat_colonias"].to_dict()
lon_lookup = df_colonias.drop_duplicates("codigo_postal").set_index("codigo_postal")["lon_colonias"].to_dict()

# Asignar lat/lon de colonia a cada escuela (como BUSCARV)
df_escuelas["lat_colonias"] = df_escuelas["codigo_postal"].map(lat_lookup)
df_escuelas["lon_colonias"] = df_escuelas["codigo_postal"].map(lon_lookup)

# Calcular distancia
df_escuelas["distancia_km"] = haversine(
    df_escuelas["lon_escuelas"], df_escuelas["lat_escuelas"],
    df_escuelas["lon_colonias"], df_escuelas["lat_colonias"]
)

# Filtrar > 10 km
df_final = df_escuelas[df_escuelas["distancia_km"] > 10]

df_final

,cct,tipo_centro,desc_tipo_centro,turno,desc_turno,servicio,desc_servicio,sostenimiento,desc_sostenimiento,nivel_educativo,...,asentamiento,cve_tipo_asentamiento,tipo_asentamiento,numero_exterior,numero_interior,lat_escuelas,lon_escuelas,lat_colonias,lon_colonias,distancia_km
3,05ACC0001O,6,ADMINISTRATIVO,400,DISCONTINUO,51,dit,11,dti,0,...,SIN COLONIA,41,NaN,NaN,NaN,25.433333,-101.000000,0.0,0.0,11110.880987
6,05ADG0002I,3,SERVICIO REGIONAL,400,DISCONTINUO,4,APOYO EN ACTIVIDADES ADMINISTRATIVAS Y DE SERV...,24,FEDERAL TRANSFERIDO,0,...,SIN COLONIA,41,NaN,NaN,NaN,25.544444,-103.441667,0.0,0.0,11353.747730
8,05ADG0004G,6,ADMINISTRATIVO,400,DISCONTINUO,4,APOYO EN ACTIVIDADES ADMINISTRATIVAS Y DE SERV...,21,ESTATAL,0,...,SIN COLONIA,41,NaN,NaN,NaN,25.428507,-100.958039,0.0,0.0,11106.726341
12,05ADG0008C,6,ADMINISTRATIVO,400,DISCONTINUO,4,APOYO EN ACTIVIDADES ADMINISTRATIVAS Y DE SERV...,24,FEDERAL TRANSFERIDO,0,...,SIN COLONIA,41,NaN,NaN,NaN,25.433333,-101.000000,0.0,0.0,11110.880987
16,05ADG0012P,6,ADMINISTRATIVO,400,DISCONTINUO,4,APOYO EN ACTIVIDADES ADMINISTRATIVAS Y DE SERV...,21,ESTATAL,0,...,SIN COLONIA,41,NaN,NaN,NaN,25.428507,-100.958039,0.0,0.0,11106.726341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13858,05PCP0005Q,9,ESCUELA,400,DISCONTINUO,9,DIPLOMADO,61,PARTICULAR,1,...,SIN COLONIA,41,NaN,NaN,NaN,25.544444,-103.441667,0.0,0.0,11353.747730
13859,05PCP0006P,9,ESCUELA,400,DISCONTINUO,9,DIPLOMADO,61,PARTICULAR,1,...,SIN COLONIA,41,NaN,NaN,NaN,25.544444,-103.441667,0.0,0.0,11353.747730
13860,05PII0002D,9,ESCUELA,400,DISCONTINUO,6,IDIOMAS,61,PARTICULAR,1,...,SIN COLONIA,41,NaN,NaN,NaN,25.544444,-103.441667,0.0,0.0,11353.747730
13862,05PII0004B,9,ESCUELA,400,DISCONTINUO,6,IDIOMAS,61,PARTICULAR,1,...,SIN COLONIA,41,NaN,NaN,NaN,25.544444,-103.441667,0.0,0.0,11353.747730


In [15]:
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[df_final["lat_escuelas"].mean(), df_final["lon_escuelas"].mean()],  zoom_start = 2, max_bounds = True)

folium.TileLayer('CartoDB positron', no_wrap = True).add_to(mapa)

min_lat = min(df_final["lat_escuelas"].min(), df_final["lat_colonias"].min())
max_lat = max(df_final["lat_escuelas"].max(), df_final["lat_colonias"].max())
min_lon = min(df_final["lon_escuelas"].min(), df_final["lon_colonias"].min())
max_lon = max(df_final["lon_escuelas"].max(), df_final["lon_colonias"].max())

mapa.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

cluster = MarkerCluster().add_to(mapa)

for _, row in df_final.iterrows():
    #Marcador para cada escuela
    folium.Marker(
        [row["lat_escuelas"], row["lon_escuelas"]],
        popup = f"Escuela - CP {row['codigo_postal']} - Distancia: {row['distancia_km']:.2f} km"
    ).add_to(cluster)

    # Marcador para la colonia
    folium.Marker(
        [row["lat_colonias"], row["lon_colonias"]],
        popup=f"Colonia - CP {row['codigo_postal']}",
        icon=folium.Icon(color="green")
    ).add_to(cluster)

    folium.PolyLine(
        locations = [
            [row["lat_colonias"], row["lon_colonias"]],
            [row["lat_escuelas"], row["lon_escuelas"]]
        ],
        color = "blue", weight = 2, opacity = 0.5
    ).add_to(mapa)

mapa.save("mapa_escuelas_distantes.html")